In [34]:
from youtube_transcript_api import YouTubeTranscriptApi
import yt_dlp

ytt_api = YouTubeTranscriptApi()

transcript = ytt_api.fetch('c4-DIbtNg54', languages = ["es"])

In [35]:

# ' '.join([t.text for t in transcript.snippets])
# transcript.snippets[0]

channel_url='https://www.youtube.com/@atemporalpodcast/videos'

ydl_opts = {
    "quiet": True,
    "extract_flat": True,   # don't download, just get metadata
    "skip_download": True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    channel_info = ydl.extract_info(channel_url, download=False)

    # Depending on URL, you may need /videos
    # info["entries"] contains all videos in the channel
    # [print(f'keys:{entry.keys()}') for entry in info["entries"]]
    # video_ids = [entry["id"] for entry in info["entries"] if entry.get("id")]
    # print(video_ids)

# len(transcript)

In [36]:
url = channel_info["entries"][0]
# channel_info["entries"][0]
# with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#     info = ydl.extract_info(url, download=False)
#     full_description = info.get("description")
# print(full_description)
transcript_text = ' '.join([snippet.text for snippet in transcript.snippets[:40]])
transcript_text

'Colombia es un país mucho más de servicios que campesino. Ahora sigue con una percepción campesina porque seguimos reforzando la gente que el futuro es estar en el campo, que la gente no, el futuro no está en el campo, que es distinto a que Colombia tenga oportunidades agroindustriales muy grandes, que es otra cosa también. Entonces, todo eso es reidentificarnos, una identidad emergente para desde ella abordar la nueva era y jugar a ganar. Alejandro Salazar tiene un doctorado en ingeniería industrial. Ha sido consultor empresarial y autor del libro La estrategia emergente. Acaba de publicar Colombia ganadora, en el que plantea una estrategia emergente para Colombia. ¿Quiere esto decir que el presidente queita este país es argentino? Creo que tenemos la mejor caricatura de Petro. Es un tipo que le busca problemas a las soluciones. Pues eso nos pasa en todos los niveles. Ha pasado, ¿cierto? No, en Bogotá nos pasa en fútbol. Nosotros no sabemos ganar. Siempre nos pasa, César, con selecci

In [29]:
from typing import List, Dict
import re

def split_spanish_sentences(text: str) -> List[str]:
    # Lightweight Spanish sentence splitter (works well enough for transcripts).
    # For higher accuracy use spaCy: es_core_news_sm
    text = re.sub(r'\s+', ' ', text).strip()
    # Split on ., !, ? followed by space and a capital letter or end of text
    parts = re.split(r'(?<=[\.\!\?])\s+(?=[A-ZÁÉÍÓÚÑÜ]|$)', text)
    # Clean tiny leftovers
    return [p.strip() for p in parts if p.strip()]

def chunk_by_sentences(
    text: str,
    target_chars: int = 1000,
    overlap_sents: int = 2,
    max_chars: int = 1400,   # soft cap to avoid giant chunks when sentences are long
) -> List[Dict]:
    sents = split_spanish_sentences(text)
    chunks = []
    buf, buf_len, start_idx = [], 0, 0

    i = 0
    while i < len(sents):
        s = sents[i]
        add_len = len(s) + (1 if buf else 0)  # +1 for space
        if buf and (buf_len + add_len > max_chars) and buf_len >= target_chars:
            # flush chunk
            chunk_text = " ".join(buf)
            chunks.append({"text": chunk_text})
            # prepare overlap
            overlap = buf[-overlap_sents:] if overlap_sents > 0 else []
            buf = overlap[:]  # start next chunk with overlap
            buf_len = len(" ".join(buf)) if buf else 0
            # do not advance i here; we’ll attempt to add s again
        else:
            # add sentence and advance
            if buf:
                buf.append(s)
                buf_len += add_len
            else:
                buf = [s]
                buf_len = len(s)
            i += 1

    # flush remainder
    if buf:
        chunks.append({"text": " ".join(buf)})

    return chunks

In [31]:
chunk_by_sentences(transcript)

[{'text': 'Colombia es un país mucho más de servicios que campesino. Ahora sigue con una percepción campesina porque seguimos reforzando la gente que el futuro es estar en el campo, que la gente no, el futuro no está en el campo, que es distinto a que Colombia tenga oportunidades agroindustriales muy grandes, que es otra cosa también. Entonces, todo eso es reidentificarnos, una identidad emergente para desde ella abordar la nueva era y jugar a ganar. Alejandro Salazar tiene un doctorado en ingeniería industrial. Ha sido consultor empresarial y autor del libro La estrategia emergente. Acaba de publicar Colombia ganadora, en el que plantea una estrategia emergente para Colombia. ¿Quiere esto decir que el presidente queita este país es argentino? Creo que tenemos la mejor caricatura de Petro. Es un tipo que le busca problemas a las soluciones. Pues eso nos pasa en todos los niveles. Ha pasado, ¿cierto? No, en Bogotá nos pasa en fútbol. Nosotros no sabemos ganar. Siempre nos pasa, César, c